In [88]:
import requests,json

In [17]:
from bs4 import BeautifulSoup

In [18]:
url = "https://www.portaltransparencia.gov.br/despesas/pagamento/"

In [19]:
r = requests.get(url+"158953"+"26405"+"2022OB800574",timeout=(3.05, 27))

In [20]:
r.status_code

200

In [21]:
soup = BeautifulSoup(r.text, "html.parser")

In [22]:
OB = {}

In [23]:
OB['numero'] = soup.select_one("body > main > div:nth-child(3) > section.dados-tabelados > div:nth-child(1) > div:nth-child(1) > span").text.strip()

In [24]:
OB['data'] = soup.select_one("body > main > div:nth-child(3) > section.dados-tabelados > div:nth-child(1) > div:nth-child(2) > span").text.strip()

In [25]:
OB['valor'] = soup.select_one("body > main > div:nth-child(3) > section.dados-tabelados > div:nth-child(3) > div.col-xs-12.col-sm-6 > span").text.strip()

In [26]:
OB['observacao'] = soup.select_one("body > main > div:nth-child(3) > section.dados-tabelados > div:nth-child(5) > div > span").text.strip()

In [27]:
OB['cnpjFavorecido'] = soup.select_one("#collapse-1 > div > div > div.col-xs-12.col-sm-3 > span > a").text.strip()

In [28]:
OB['nomeFavorecido'] = soup.select_one("#collapse-1 > div > div > div.col-xs-12.col-sm-9 > span").text.strip()

In [29]:
OB['processo'] = soup.select_one("body > main > div:nth-child(3) > section:nth-child(4) > div.bloco > div > div > span").text.strip()

In [78]:
url_curl = f'https://www.portaltransparencia.gov.br/despesas/documento/documentos-relacionados/resultado?paginacaoSimples=true&tamanhoPagina=15&offset=0&direcaoOrdenacao=asc&colunaOrdenacao=fase&colunasSelecionadas=data%2Cfase%2CdocumentoResumido%2Cespecie&fase=Empenho&codigo=158953264052021NE000116&_=1656178891541'

In [79]:
get_curl_data(url_curl)

{'draw': 0,
 'recordsTotal': 3,
 'recordsFiltered': 3,
 'data': [{'data': '31/12/2021',
   'flgExisteDocumentoRelacionadoNoDM': True,
   'fase': 'LiquidaÃ§Ã£o',
   'documento': '158953264052021NS003516',
   'documentoResumido': '2021NS003516',
   'especie': '',
   'orgaoSuperior': 'MinistÃ©rio da EducaÃ§Ã£o',
   'orgaoVinculado': 'Instituto Federal do CearÃ¡',
   'unidadeGestora': 'CAMPUS TABULEIRO DO NORTE/IFCE',
   'elementoDespesa': 'MÃºltiplo',
   'favorecido': 'Sem informaÃ§Ã£o',
   'valor': ''},
  {'data': '24/05/2022',
   'flgExisteDocumentoRelacionadoNoDM': True,
   'fase': 'LiquidaÃ§Ã£o',
   'documento': '158953264052022NS001193',
   'documentoResumido': '2022NS001193',
   'especie': '',
   'orgaoSuperior': 'MinistÃ©rio da EducaÃ§Ã£o',
   'orgaoVinculado': 'Instituto Federal do CearÃ¡',
   'unidadeGestora': 'CAMPUS TABULEIRO DO NORTE/IFCE',
   'elementoDespesa': 'Material, Bem ou ServiÃ§o para DistribuiÃ§Ã£o Gr',
   'favorecido': 'REIS INDUSTRIA E COMERCIO DE BOLSAS E PROMOCIO

In [134]:
url_pagamentos = "https://www.portaltransparencia.gov.br/despesas/documento/documentos-relacionados/resultado?paginacaoSimples=true&tamanhoPagina=15&offset=0&direcaoOrdenacao=asc&colunaOrdenacao=fase&colunasSelecionadas=data%2Cfase%2CdocumentoResumido%2Cespecie&fase=Empenho&codigo=158953264052021NE000116"

In [135]:
get_pagamentos(url_pagamentos)

[{'documento': '2022OB800574',
  'data': '01/06/2022',
  'favorecido': 'REIS INDUSTRIA E COMERCIO DE BOLSAS E PROMOCIONAIS EIRE',
  'valor': '40.100,00'}]

In [131]:
def get_pagamentos(url):
    import json

    pagamentos = []
    r = requests.get(url)
    if r.status_code == 200:
        try:
            documentosRelacionados = json.loads(r.text)
            if len(documentosRelacionados['data']) > 0:
                documentos = documentosRelacionados['data']
                for documento in documentos:
                    if documento['fase'] == 'Pagamento':
                        pagamentos.append(
                            {
                                'documento': documento['documentoResumido'],
                                'data': documento['data'],
                                'favorecido': documento['favorecido'],
                                'valor': documento['valor']
                            }
                        )
        except Exception as erro:
            return {"erro": erro}
    else:
        return {"erro": r.status_code}

    return pagamentos

In [58]:
def get_curl_data(url):
    import pycurl
    import certifi
    import json
    from io import BytesIO

    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.WRITEDATA, buffer)
    c.setopt(c.CAINFO, certifi.where())
    c.perform()
    c.close()

    body = buffer.getvalue()
    # Body is a byte string.
    # We have to know the encoding in order to print it to a text file
    # such as standard output.
    return json.loads(body.decode('iso-8859-1'))